In [1]:
import requests as req
import pandas as pd
import sqlite3 as sql
import requests
from functools import reduce
import numpy as np
import os.path
from os import path

In [2]:
sqlite_file = 'salaries_data.sql'

In [3]:
db = sql.connect(":memory:")

In [4]:
sql_file = open(sqlite_file)
sql_as_string = sql_file. read()
db. executescript(sql_as_string)

## Data  Manipulation

In [5]:
salary_query= """
SELECT 
 firstname, lastname, department, salary, position
FROM Salaries
"""

In [6]:
salaries = pd.read_sql(salary_query, db)

In [7]:
salaries.head()

,firstname,lastname,department,salary,position
0,DJ,Durkin,PRES,881010.96,Head Coach
1,Darryll J,Pines,PRES,765000.00,President
2,Wallace,Loh,PRES,734565.88,Spec Asst to the Pres
3,Mark,Turgeon,PRES,723312.99,Head Coach
4,Brenda S,Frese,PRES,659386.43,Head Coach BB


In [8]:
def count_nonempty(d): 
    total = 0
    for k in d.keys():
        if (d.get(k) != None):
            total += 1
    print("the non empty total", total)

In [38]:
headers = {
  'Accept': 'application/json'
}

In [39]:
def fetch_profs(): 


    profs = dict()
    matched_profs = dict()

    for i in range(0, 10000, 1000):
        r = requests.get('https://api.planetterp.com/v1/professors?reviews=true', params={
          'limit': '1000',
          'offset': i,
        }, headers = headers)

        for e in r.json():
            profs[e.get("slug")] = e
            matched_profs[e.get("slug")] = None

    
    print(len(profs.values()))
    return (profs, matched_profs)

In [40]:
def find_closest_match(name, matches):
    firstname = name.split(" ")[0]
    ret = [ m  for m in matches if m[0].find(firstname) >= 0 ]
    if(len(ret) > 0):
        return ret[0]
    return None
def calc_avg_review(revs):
    length = len(revs)
    if(length == 0):
        return np.nan
    total = 0
    for r in revs:
        total += r.get("rating")
    return total / length
        

In [41]:
def get_department(dep, profs, matched_profs):
    matches = dict()

    for prof in profs.values():
        
        slug = prof.get("slug")
        name = prof.get("name")
        courses = prof.get("courses")
        avg_rating = calc_avg_review(prof.get("reviews"))
        
        if(matched_profs.get(slug) != None):
            continue

        lastname = name.split(" ")[-1]

        salary_query= f"""
            SELECT 
            firstname, 
            lastname,
            department,
            salary,
            position
            FROM Salaries
            WHERE lastname like "{lastname}"
            AND department = "{dep}"
        """
        rows = list(db.execute(salary_query))
        matches[lastname] = (len(rows), rows, name, slug, courses, avg_rating)
        

    for match in matches.keys():
        (length, v, name, slug, courses, avg_rating) = matches.get(match)
        if(length > 0):
           
            value = find_closest_match(name, v)
         
            if(value != None):
                
                matched_profs[slug] = (
                    value[0],
                    value[1],
                    value[2],
                    value[3],
                    value[4],
                    avg_rating,
                    slug,
                    name,
                    "-".join(courses)
                )


    return

In [42]:
data = []

In [43]:
def fetch_matched_salaries():
    if(path.isfile("matched_salaries.csv")):
        return pd.read_csv("matched_salaries.csv")
    
    
    (profs, matched_profs) = fetch_profs()
    
    arr = list(set(salaries["department"]))
    for dep in arr:   
        get_department(dep, profs, matched_profs)
        
    data = [ list(v)  for v in matched_profs.values() if v != None]
    matched_salaries = pd.DataFrame(data, columns=['firstname', 'lastname', 'department', 'salary', "position", "avg_rating", 'slug', "planetTerpName", "courses"])
    matched_salaries.to_csv(path_or_buf="matched_salaries.csv", index=False)
    return matched_salaries


In [44]:
matched_salaries = fetch_matched_salaries()
matched_salaries.head()

,firstname,lastname,department,salary,position,avg_rating,slug,planetTerpName,courses
0,A W,Kruglanski,BSOS,322501.74,Distinguished Univ Professor,2.0,kruglanski,A Kruglanski,PSYC743-PSYC748M-PSYC489H-PSYC489T-PSYC789-PSY...
1,Aaron,Finkle,BSOS,75054.05,Lecturer,3.5,finkle,Aaron Finkle,ECON306-ECON641-ECON321-ECON456-ECON488K-BUSO604
2,Aaron Gilad,Kusne,ENGR,17945.31,Lecturer,NaN,kusne,Aaron Kusne,ENMA489L
3,Aaron Wolf,Mannes,PLCY,15300.00,Lecturer,NaN,mannes,Aaron Mannes,PLCY780
4,Aaron George,Swanlek,ARHU,37087.20,Lecturer,5.0,swanlek,Aaron Swanlek,COMM107


In [50]:
def fetch_reviewsData(_data):
    if(path.isfile("reviews.csv")):
        return pd.read_csv("reviews.csv")
        
    reviews_data = []
    for v in _data:

        r = requests.get('https://api.planetterp.com/v1/professor', params={
          'name': v,  'reviews': 'true'
        }, headers = headers).json()


        slug = r.get("slug")
        revs = r.get("reviews")
        if(revs == None):
            continue

        for rev in revs:  

            reviews_data.append([ 
                slug,
                rev.get("rating"),  
                rev.get("expected_grade"), 
                rev.get("course"),
                rev.get("review"),
            ])   
    reviews_df = pd.DataFrame(reviews_data, columns=['slug', 'rating', 'expected_grade', 'course', 'review'])
    reviews_df.to_csv(path_or_buf="reviews.csv", index=False)
    return  reviews_df



In [ ]:
reviews_df = fetch_reviewsData(list(matched_salaries["planetTerpName"]))
reviews_df.head()

In [47]:
# reviews_df.to_csv(path_or_buf="reviews.csv", index=False)

In [ ]:
data

In [ ]:
# word vector classifications - gensim

# word cloud with reviews (bag of words)
    # words that describe popular profesors
    
# scatter plot of professor salaries vs ratings
# scatter plot of depart salaries vs ratings

# consider the title of the professors which may be a reason for increased salary


## EDA

In [27]:
dept_avg = matched_salaries.groupby("department").mean().sort_values(by=["salary"], ascending=False)
dept_avg

,salary
department,
VPR,166288.755000
GRAD,164923.610000
BMGT,161424.326644
DIT,156318.315833
PRES,154999.928333
BSOS,118920.834279
CMNS,117202.848151
SVPAAP,111728.427308
ENGR,111355.564150
